In [ ]:
!pip install gradio
!pip install nemo_toolkit[nlp] nemo_toolkit[asr] nemo_toolkit[tts]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 74.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.9/91.9 kB 9.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.6/313.6 kB 34.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 41.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.2/47.2 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 9.3 MB/s et

In [ ]:
!pip install librosa
!pip install waveglow
!pip install numpy scipy librosa unidecode inflect librosa

In [ ]:
import torch
import nemo.collections.asr as nemo_asr
import nemo.collections.tts as nemo_tts
from nemo.collections.tts.models import Tacotron2Model, WaveGlowModel
from transformers import pipeline,AutoModelForCausalLM, AutoTokenizer
import gradio as gr
import numpy as np
import base64
import soundfile as sf
import librosa
from scipy.io.wavfile import write

[NeMo W 2024-04-24 16:57:14 nemo_logging:349] /usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
      self.pid = os.fork()
    


In [ ]:
print("Initializing ASR model...")
asr_model = nemo_asr.models.EncDecHybridRNNTCTCBPEModel.from_pretrained(model_name="nvidia/stt_en_fastconformer_hybrid_large_streaming_multi")
print("ASR model initialized.")

print("Initializing TTS model...")
tts_model = nemo_tts.models.Tacotron2Model.from_pretrained(model_name="Tacotron2-22050Hz")
print("TTS model initialized.")

waveglow_model = WaveGlowModel.from_pretrained(model_name="WaveGlow-22050Hz").cuda().eval()

print("Initializing sentiment analysis pipeline...")
emotion_model = pipeline("sentiment-analysis")
print("Sentiment analysis pipeline initialized.")

emotion_model = pipeline("sentiment-analysis")
print("Sentiment analysis pipeline initialized.")

# Initialize DialoGPT model
dialogpt_tokenizer = AutoTokenizer.from_pretrained("microsoft/DialoGPT-large")
dialogpt_model = AutoModelForCausalLM.from_pretrained("microsoft/DialoGPT-large")

Initializing ASR model...


[NeMo W 2024-04-24 16:57:25 nemo_logging:349] /usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
    The secret `HF_TOKEN` does not exist in your Colab secrets.
    To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
    You will be able to reuse this secret in all of your notebooks.
    Please note that authentication is recommended but still optional to access public models or datasets.
      warnings.warn(
    


(…)former_hybrid_large_streaming_multi.nemo:   0%|          | 0.00/460M [00:00<?, ?B/s]

[NeMo I 2024-04-24 16:57:59 mixins:172] Tokenizer SentencePieceTokenizer initialized with 1024 tokens


[NeMo W 2024-04-24 16:58:00 modelPT:165] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath:
    - - /raid/local//bucket1/tarred_audio_manifest.json
    - - /raid/local//bucket2/tarred_audio_manifest.json
    - - /raid/local//bucket3/tarred_audio_manifest.json
    - - /raid/local//bucket4/tarred_audio_manifest.json
    - - /raid/local//bucket5/tarred_audio_manifest.json
    - - /raid/local//bucket6/tarred_audio_manifest.json
    - - /raid/local//bucket7/tarred_audio_manifest.json
    - - /raid/local//bucket8/tarred_audio_manifest.json
    sample_rate: 16000
    batch_size: 1
    shuffle: true
    num_workers: 4
    pin_memory: true
    use_start_end_token: false
    trim_silence: false
    max_duration: 25
    min_duration: 0.1
    is_tarred: true
    tarred_audio_filepaths:
    - - /data2/nemo_asr/nemo_asr_set_3.0//bucket1/audi

...

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/642 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.75G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [ ]:
def get_weather_answer(audio_file):
    sr,audio = audio_file
    original_audio_file = '/content/uploaded_audio.wav'  # Path to save the uploaded audio
    sf.write(original_audio_file, audio, sr)

    # original_audio_file = '/content/Recording (2).m4a'
    audio, sr = librosa.load(original_audio_file, sr=None, mono=False)

    if audio.ndim > 1:
        audio = librosa.to_mono(audio)

    if sr != 22050:
        audio = librosa.resample(audio, orig_sr=sr, target_sr=22050)
        sr = 22050

    processed_audio_file = '/content/processed_audio.wav'
    sf.write(processed_audio_file, audio, sr)

    transcribed_text = asr_model.transcribe([processed_audio_file])[0]
    print(transcribed_text)
    print(dialogpt_tokenizer.eos_token)

    new_user_input = transcribed_text[0] + dialogpt_tokenizer.eos_token
    new_user_input_ids = dialogpt_tokenizer.encode(new_user_input, return_tensors='pt')
    chat_history_ids = dialogpt_model.generate(new_user_input_ids, max_length=1000, pad_token_id=dialogpt_tokenizer.eos_token_id)

    with torch.no_grad():
        response = dialogpt_tokenizer.decode(chat_history_ids[:, new_user_input_ids.shape[-1]:][0], skip_special_tokens=True)
        print('Valasz',response)
        sentiment = emotion_model(response)[0]['label']
        print('Sentiment:',sentiment)
        tokens = tts_model.parse(response)
        mel_outputs = tts_model.generate_spectrogram(tokens=tokens)

    # Generate audio waveform from mel spectrogram using WaveGlow
    with torch.no_grad():
        audio_waveform = waveglow_model.convert_spectrogram_to_audio(spec=mel_outputs)

    # Convert audio waveform tensor to numpy array and get sample rate
    audio_waveform_np = audio_waveform.squeeze().cpu().numpy()

    # Save the synthesized audio
    sf.write("synthesized_audio.wav", audio_waveform_np, samplerate=22050)
    text_output_with_snetiment = response + ' The sentiment is: ' + sentiment

    return text_output_with_snetiment, '/content/synthesized_audio.wav'

print("Creating Gradio interface...")
audio_input = gr.Audio(label="Upload Audio File")
text_output = gr.Textbox(label="Answer")
audio_output = gr.Audio(label="Synthesized Audio")

gr.Interface(get_weather_answer, audio_input, [text_output, audio_output]).launch(debug=True)

Creating Gradio interface...
Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://119ab355acf56da9b2.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


['i am happy']
<|endoftext|>
Valasz I am happy too
Sentiment: POSITIVE


Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

...

A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


['there is a fire']
<|endoftext|>
Valasz I think it's a fire.
Sentiment: NEGATIVE


Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


['is it training']
<|endoftext|>
Valasz I think it's a training for a new character.
Sentiment: NEGATIVE
Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://119ab355acf56da9b2.gradio.live


In [ ]:
print("Initializing ASR model...")
asr_model = nemo_asr.models.EncDecHybridRNNTCTCBPEModel.from_pretrained(model_name="nvidia/stt_en_fastconformer_hybrid_large_streaming_multi")
print("ASR model initialized.")

print("Initializing TTS model...")
tts_model = nemo_tts.models.Tacotron2Model.from_pretrained(model_name="Tacotron2-22050Hz")
print("TTS model initialized.")

waveglow_model = WaveGlowModel.from_pretrained(model_name="WaveGlow-22050Hz").cuda().eval()

print("Initializing sentiment analysis pipeline...")
emotion_model = pipeline("sentiment-analysis")
print("Sentiment analysis pipeline initialized.")


def get_weather_answer(audio_file):
    # Path to the original audio file
    original_audio_file = '/content/Rögzítés-_3_.wav'

    # Load the audio file
    audio, sr = librosa.load(original_audio_file, sr=None, mono=False)

    # Convert stereo to mono if needed
    if audio.ndim > 1:
        audio = librosa.to_mono(audio)

    # Resample to 16 kHz if needed
    if sr != 22050:
        audio = librosa.resample(audio, orig_sr=sr, target_sr=22050)
        sr = 22050

    # Save the processed audio file
    processed_audio_file = '/content/processed_audio.wav'
    sf.write(processed_audio_file, audio, sr)

    transcribed_text = asr_model.transcribe([processed_audio_file])[0]
    print(transcribed_text)

    if "weather" in transcribed_text[0].lower() and "today" in transcribed_text[0].lower():
        weather_condition = "raining"

        if weather_condition == "raining":
            answer = "The weather is still raining."
        else:
            answer = "The weather is not raining."

        sentiment = emotion_model(answer)[0]['label']
        answer += f" The sentiment is {sentiment}."
        print(sentiment)
        print(answer)

        with torch.no_grad():
            tokens = tts_model.parse(answer)
            mel_outputs = tts_model.generate_spectrogram(tokens=tokens)

        # Generate audio waveform from mel spectrogram using WaveGlow
        with torch.no_grad():
            audio_waveform = waveglow_model.convert_spectrogram_to_audio(spec=mel_outputs)

            # Convert audio waveform tensor to numpy array and get sample rate
        audio_waveform_np = audio_waveform.squeeze().cpu().numpy()
        print('audio waveform',audio_waveform_np.shape)

        sf.write("synthesized_audio.wav", audio_waveform_np, samplerate=22050)
    else:
        answer = "Sorry, I can only provide information about the weather today."
        audio_output = None

    return answer, audio_output

print("Creating Gradio interface...")
audio_input = gr.Audio(label="Upload Audio File")
text_output = gr.Textbox(label="Answer")
audio_output = gr.Audio(label="Synthesized Audio")

gr.Interface(get_weather_answer, audio_input, [text_output, audio_output]).launch(debug=True)

Initializing ASR model...
[NeMo I 2024-04-23 21:58:13 mixins:172] Tokenizer SentencePieceTokenizer initialized with 1024 tokens


[NeMo W 2024-04-23 21:58:14 modelPT:165] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath:
    - - /raid/local//bucket1/tarred_audio_manifest.json
    - - /raid/local//bucket2/tarred_audio_manifest.json
    - - /raid/local//bucket3/tarred_audio_manifest.json
    - - /raid/local//bucket4/tarred_audio_manifest.json
    - - /raid/local//bucket5/tarred_audio_manifest.json
    - - /raid/local//bucket6/tarred_audio_manifest.json
    - - /raid/local//bucket7/tarred_audio_manifest.json
    - - /raid/local//bucket8/tarred_audio_manifest.json
    sample_rate: 16000
    batch_size: 1
    shuffle: true
    num_workers: 4
    pin_memory: true
    use_start_end_token: false
    trim_silence: false
    max_duration: 25
    min_duration: 0.1
    is_tarred: true
    tarred_audio_filepaths:
    - - /data2/nemo_asr/nemo_asr_set_3.0//bucket1/audi

[NeMo I 2024-04-23 21:58:14 features:289] PADDING: 0


[NeMo W 2024-04-23 21:58:15 nemo_logging:349] /usr/local/lib/python3.10/dist-packages/torch/nn/modules/rnn.py:83: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
      warnings.warn("dropout option adds dropout after all but last "
    


[NeMo I 2024-04-23 21:58:15 rnnt_models:217] Using RNNT Loss : warprnnt_numba
    Loss warprnnt_numba_kwargs: {'fastemit_lambda': 0.005, 'clamp': -1.0}
[NeMo I 2024-04-23 21:58:16 save_restore_connector:249] Model EncDecHybridRNNTCTCBPEModel was successfully restored from /root/.cache/huggingface/hub/models--nvidia--stt_en_fastconformer_hybrid_large_streaming_multi/snapshots/61f9be786c1280e97468291d254dc2599803ebc0/stt_en_fastconformer_hybrid_large_streaming_multi.nemo.
ASR model initialized.
Initializing TTS model...
[NeMo I 2024-04-23 21:58:16 cloud:58] Found existing object /root/.cache/torch/NeMo/NeMo_1.23.0/tts_en_tacotron2/319a832e78fac50399d90aee0f083927/tts_en_tacotron2.nemo.
[NeMo I 2024-04-23 21:58:16 cloud:64] Re-using file from: /root/.cache/torch/NeMo/NeMo_1.23.0/tts_en_tacotron2/319a832e78fac50399d90aee0f083927/tts_en_tacotron2.nemo
[NeMo I 2024-04-23 21:58:16 common:924] Instantiating model from pre-trained checkpoint


...

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

[NeMo W 2024-04-23 21:59:24 tacotron2:144] parse() is meant to be called in eval mode.


['what is the weather today']
NEGATIVE
The weather is still raining. The sentiment is NEGATIVE.
audio waveform (82688,)


Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/gradio/queueing.py", line 527, in process_events
    response = await route_utils.call_process_api(
  File "/usr/local/lib/python3.10/dist-packages/gradio/route_utils.py", line 261, in call_process_api
    output = await app.get_blocks().process_api(
  File "/usr/local/lib/python3.10/dist-packages/gradio/blocks.py", line 1788, in process_api
    result = await self.call_function(
  File "/usr/local/lib/python3.10/dist-packages/gradio/blocks.py", line 1340, in call_function
    prediction = await anyio.to_thread.run_sync(
  File "/usr/local/lib/python3.10/dist-packages/anyio/to_thread.py", line 33, in run_sync
    return await get_asynclib().run_sync_in_worker_thread(
  File "/usr/local/lib/python3.10/dist-packages/anyio/_backends/_asyncio.py", line 877, in run_sync_in_worker_thread
    return await future
  File "/usr/local/lib/python3.10/dist-packages/anyio/_backends/_asyncio.py", line 807, in run
    r

Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://a929cb38dbdfdd7da9.gradio.live
